# SILERO TTS

In [ ]:
# !mkdir src
# !touch src/test.txt

# !echo "Я приближался к месту моего назначения. " >> src/test.txt
# !echo "Вокруг меня, прижимаясь к самой дороге, зеленел лес, изредка уступая место полянам, поросшим желтой осокою. " >> src/test.txt
# !echo "Солнце садилось уже который час, все никак не могло сесть и висело низко над горизонтом. " >> src/test.txt
# !echo "Машина катилась по узкой дороге, засыпанной хрустящим гравием. " >> src/test.txt
# !echo "Крупные камни я пускал под колесо, и каждый раз в багажнике лязгали и громыхали пустые канистры." >> src/test.txt

In [1]:
!mkdir src
!unzip src.zip -d ./src

Archive:  src.zip
  inflating: ./src/part001 - Том 1..txt  
  inflating: ./src/part002 - Пролог..txt  
  inflating: ./src/part003 - Глава 1..txt  
  inflating: ./src/part004 - Глава 2..txt  
  inflating: ./src/part005 - Глава 3..txt  
  inflating: ./src/part006 - Глава 4..txt  
  inflating: ./src/part007 - Глава 5..txt  


In [2]:
!git clone https://github.com/fnx4/silero-tts silero

Cloning into 'silero'...
remote: Enumerating objects: 221, done.
remote: Counting objects: 100% (221/221), done.
remote: Compressing objects: 100% (128/128), done.
remote: Total 221 (delta 114), reused 175 (delta 68), pack-reused 0 (from 0)
Receiving objects: 100% (221/221), 111.84 KiB | 874.00 KiB/s, done.
Resolving deltas: 100% (114/114), done.


In [3]:
!python --version
!ffmpeg -version | grep "ffmpeg version"
!nvidia-smi

Python 3.12.11
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
Fri Aug 29 10:09:12 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.77.01              Driver Version: 566.36         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090        On  |   00000000:01:00.0  On |                  Off |
| 46%   42C    P0             61W /  360W |    1078MiB /  24564MiB |      4%      Default |
|                                        

In [4]:
!git clone https://github.com/RVC-Project/Retrieval-based-Voice-Conversion-WebUI rvc
!git --work-tree=rvc --git-dir=rvc/.git reset --hard 64b78bed3b48f595e27fbe302819b2df13124095
!pip install virtualenv
!virtualenv -p /usr/bin/python3.10 rvc/venv
!rvc/venv/bin/python -m pip install --force-reinstall pip==24.0
!rvc/venv/bin/python -m pip install torch==2.5.1+cu124 torchaudio==2.5.1+cu124 torchvision==0.20.1+cu124 --index-url https://download.pytorch.org/whl/cu124
!rvc/venv/bin/python -m pip install --quiet -r rvc/requirements.txt

!wget -q --show-progress -O rvc/assets/rmvpe/rmvpe.pt https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/rmvpe.pt
!wget -q --show-progress -O rvc/assets/hubert/hubert_base.pt https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/hubert_base.pt

Cloning into 'rvc'...
remote: Enumerating objects: 4593, done.
remote: Total 4593 (delta 0), reused 0 (delta 0), pack-reused 4593 (from 1)
Receiving objects: 100% (4593/4593), 14.33 MiB | 7.91 MiB/s, done.
Resolving deltas: 100% (2883/2883), done.
HEAD is now at 64b78be 20240604
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 11.0 MB/s eta 0:00:00
created virtual environment CPython3.10.12.final.0-64 in 304ms
  creator CPython3Posix(dest=/content/rvc/venv, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==25.2, setuptools==80.9.0
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.9 MB/s  0:00:00
  Attempting uninstall: pip
    Found existi

In [5]:
!wget -q --show-progress https://huggingface.co/fnx/RVCv2-RMVPE-ru-saya/resolve/main/ru-saya-1000.zip
!unzip ru-saya-1000.zip -d rvc/assets/weights
!mv rvc/assets/weights/ru-saya/* rvc/assets/weights

ru-saya-1000.zip    100%[===================>]  58.01M  12.3MB/s    in 5.1s    
Archive:  ru-saya-1000.zip
   creating: rvc/assets/weights/ru-saya/
  inflating: rvc/assets/weights/ru-saya/ru-saya-1000.pth  
  inflating: rvc/assets/weights/ru-saya/trained_IVF3468_Flat_nprobe_1_ru-saya_v2.index  


In [6]:
!apt-get -qq install pandoc

#!pip install -r silero/requirements-local.txt --quiet
!pip install -r silero/requirements.txt --quiet

Selecting previously unselected package libcmark-gfm0.29.0.gfm.3:amd64.
(Reading database ... 126371 files and directories currently installed.)
Preparing to unpack .../libcmark-gfm0.29.0.gfm.3_0.29.0.gfm.3-3_amd64.deb ...
Unpacking libcmark-gfm0.29.0.gfm.3:amd64 (0.29.0.gfm.3-3) ...
Selecting previously unselected package libcmark-gfm-extensions0.29.0.gfm.3:amd64.
Preparing to unpack .../libcmark-gfm-extensions0.29.0.gfm.3_0.29.0.gfm.3-3_amd64.deb ...
Unpacking libcmark-gfm-extensions0.29.0.gfm.3:amd64 (0.29.0.gfm.3-3) ...
Selecting previously unselected package pandoc-data.
Preparing to unpack .../pandoc-data_2.9.2.1-3ubuntu2_all.deb ...
Unpacking pandoc-data (2.9.2.1-3ubuntu2) ...
Selecting previously unselected package pandoc.
Preparing to unpack .../pandoc_2.9.2.1-3ubuntu2_amd64.deb ...
Unpacking pandoc (2.9.2.1-3ubuntu2) ...
Setting up libcmark-gfm0.29.0.gfm.3:amd64 (0.29.0.gfm.3-3) ...
Setting up libcmark-gfm-extensions0.29.0.gfm.3:amd64 (0.29.0.gfm.3-3) ...
Setting up pandoc-da

In [7]:
%run silero/silero_tts.py -t 16 -s xenia -d cuda -r 48000 --merge --rvc --rvc_model_pth ru-saya-1000.pth -i ./src -o silero/output_raw


Namespace(input='./src', output='silero/output_raw', threads='16', speaker='xenia', device='cuda', rate='48000', ignore_newlines=False, merge=True, rvc=True, rvc_model_pth='ru-saya-1000.pth', rvc_model_index=None, rvc_transpose='3')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
100%|██████████| 59.0M/59.0M [00:06<00:00, 10.3MB/s]



file: ./src/part001 - Том 1..txt ...


 CHAPTER:   0%|          | 0/1 [00:00<?, ?it/s]

    TEXT:   0%|          | 0/1 [00:00<?, ?it/s]

...

SENTENCE:   0%|          | 0/88 [00:00<?, ?it/s]


Merging and encoding, please wait...


 RVC:   0%|          | 0/7 [00:00<?, ?it/s]

OPUS:   0%|          | 0/7 [00:00<?, ?it/s]


Finished!


In [8]:
!zip -r output.zip silero/output_raw/_opus | sort
!ls -l output.zip

  adding: silero/output_raw/_opus/part001_-_Том_1__000000.opus (deflated 0%)
  adding: silero/output_raw/_opus/part002_-_Пролог__000000.opus (deflated 0%)
  adding: silero/output_raw/_opus/part003_-_Глава_1__000000.opus (deflated 0%)
  adding: silero/output_raw/_opus/part004_-_Глава_2__000000.opus (deflated 0%)
  adding: silero/output_raw/_opus/part005_-_Глава_3__000000.opus (deflated 0%)
  adding: silero/output_raw/_opus/part006_-_Глава_4__000000.opus (deflated 0%)
  adding: silero/output_raw/_opus/part007_-_Глава_5__000000.opus (deflated 0%)
  adding: silero/output_raw/_opus/ (stored 0%)
-rw-r--r-- 1 root root 8358493 Aug 29 10:19 output.zip


In [ ]:
# from IPython.display import Audio
# wav_folder = "silero/output_raw/_wav"
# for file in sorted(os.listdir(wav_folder)):
#     print(file)
#     display(Audio(os.path.join(wav_folder, file)))